# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta1

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills and requires Orchestration

In [3]:
// Load native skill into the kernel registry, sharing its functions with prompt templates
var planner = new SequentialPlanner(kernel);

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [5]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "skills");
kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory, "SummarizeSkill");
kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory, "WriterSkill");

Define your ASK. What do you want the Kernel to do?

In [6]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(JsonSerializer.Serialize(originalPlan, new JsonSerializerOptions { WriteIndented = true }));

Original plan:

{
  "state": [],
  "steps": [
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Valentine\u0027s day date ideas"
        }
      ],
      "outputs": [
        "DATE_IDEAS"
      ],
      "next_step_index": 0,
      "name": "Brainstorm",
      "plugin_name": "WriterSkill",
      "description": "Given a goal or topic description generate a list of ideas"
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$DATE_IDEAS[0]"
        }
      ],
      "outputs": [
        "RESULT__POEM_1"
      ],
      "next_step_index": 0,
      "name": "ShortPoem",
      "plugin_name": "WriterSkill",
      "description": "Turn a scenario into a short and entertaining poem."
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$DATE_IDEAS[1]"
        }
      ],
      "out

As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the skills that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next skill.

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [9]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(
    skPrompt, 
    "Shakespeare", 
    "ShakespeareSkill",
    requestSettings: new OpenAIRequestSettings()
    {
        MaxTokens = 2000,
        Temperature = 0.2,
        TopP = 0.5,
    });

Let's update our ask using this new skill.

In [10]:
var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(JsonSerializer.Serialize(newPlan, new JsonSerializerOptions { WriteIndented = true }));

Updated plan:

{
  "state": [],
  "steps": [
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Valentine\u0027s Day date ideas"
        }
      ],
      "outputs": [
        "DATE_IDEAS"
      ],
      "next_step_index": 0,
      "name": "Brainstorm",
      "plugin_name": "WriterSkill",
      "description": "Given a goal or topic description generate a list of ideas"
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$DATE_IDEAS"
        }
      ],
      "outputs": [
        "POEM"
      ],
      "next_step_index": 0,
      "name": "ShortPoem",
      "plugin_name": "WriterSkill",
      "description": "Turn a scenario into a short and entertaining poem."
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "$POEM"
        }
      ],
      "outputs": [
        "SHAKE

### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [14]:
var originalPlanResult = await originalPlan.InvokeAsync(kernel.CreateNewContext());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.GetValue<string>(), 100));

Original Plan results:

Here's a limerick for the event "Romantic dinner at a fancy restaurant":

There once was a couple so
sweet,
Who planned a fancy dinner to eat,
But the food was so small,
They left feeling quite tall,
And ended up stopping for street meat.
Romantic dinner at a fancy restaurant,
The ambiance is quite
elegant,
But beware of the food,
It might be overpriced and crude,
So stick to the bread and butter,
my friend.

Picnic in the park with homemade food and wine,
A perfect way to spend some time,
But
Romantic dinner at a fancy restaurant,
The ambiance was quite elegant,
But the food was so small,
We
left feeling quite appalled,
Next time we'll stick to pizza and a tent.

Picnic in the park with
homemade food and wine,
We thought it would be just divine,
But the



Now lets execute and print the new plan:

In [16]:
var newPlanResult = await kernel.RunAsync(newPlan);
Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult.GetValue<string>());

New Plan results:

The first stanza warns about the possibility of ants ruining a romantic picnic in the park. The second stanza suggests a cooking class as a fun and educational date idea.
